# kuso-agent を Google Colab で起動する

このノートブックを上から順番に実行すると、`uv run adk web` で立ち上がる kuso-agent を Colab 上で試せます。

**事前に用意するもの**

- [Google AI Studio](https://aistudio.google.com/api-keys) で取得した Gemini API キー (`GOOGLE_API_KEY`)
  - Googleアカウントさえあればキーは発行できます
- GitHub の公開リポジトリ `https://github.com/i-icc/kuso-agent.git` (Colab からそのまま clone できます)
- 標準の Python 3 ランタイム (GPU は不要で CPU だけで十分)


## 1. 作業ディレクトリとリポジトリ URL

必要に応じて `REPO_URL` や `PROJECT_NAME` を書き換えてから実行してください。

In [ ]:
import os
import pathlib

PROJECT_NAME = "kuso-agent"
REPO_URL = "https://github.com/i-icc/kuso-agent.git"
WORKDIR = f"/content/{PROJECT_NAME}"

pathlib.Path(WORKDIR).parent.mkdir(parents=True, exist_ok=True)
os.environ["PROJECT_NAME"] = PROJECT_NAME
os.environ["REPO_URL"] = REPO_URL
os.environ["WORKDIR"] = WORKDIR

print("Workspace directory:", WORKDIR)
print("Repository URL:", REPO_URL)


## 2. リポジトリを取得

古いクローンを削除してから最新の main ブランチを取得します。

In [ ]:
%%bash
set -eux
rm -rf "$WORKDIR"
git clone "$REPO_URL" "$WORKDIR"
ls "$WORKDIR"


## 3. uv をインストールして依存関係を同期

`uv sync --frozen --no-dev` で `pyproject.toml` に沿って依存関係を揃えます。初回は 1〜2 分かかる場合があります。

In [ ]:
%%bash
set -eux
python -m pip install -U pip
python -m pip install -U "uv>=0.5.9"
cd "$WORKDIR"
uv sync --frozen --no-dev


## 4. `.env` に Gemini API キーを保存

実行後に表示されるプロンプトへ API キーを貼り付けます。値はログに表示されないので安心してください。

In [ ]:
import getpass
import json
import os
import pathlib

workdir = pathlib.Path(os.environ["WORKDIR"])
env_path = workdir / ".env"
api_key = getpass.getpass("Gemini API key (入力は画面に表示されません): ").strip()
if not api_key:
    raise ValueError("Gemini API key is required.")

env_lines = [
    "GOOGLE_GENAI_USE_VERTEXAI=false",
    f"GOOGLE_API_KEY={json.dumps(api_key)}",
]
env_path.write_text("\n".join(env_lines) + "\n", encoding="utf-8")
print(f".env を作成しました: {env_path}")


## 5. サーバーを起動 (`adk web`)

既にサーバーが動いている場合は一度停止してから再起動します。ログがこのセルにストリームされます。

In [ ]:
import os
import subprocess
import threading

PORT = int(os.environ.get("PORT", "8000"))
ENV = os.environ.copy()
ENV["PORT"] = str(PORT)
CMD = [
    "uv",
    "run",
    "adk",
    "web",
    "--host",
    "0.0.0.0",
    "--port",
    str(PORT),
]

def _stop_running(proc):
    if proc and proc.poll() is None:
        print("既存のサーバーを停止します...", flush=True)
        proc.terminate()
        try:
            proc.wait(timeout=10)
        except subprocess.TimeoutExpired:
            proc.kill()

if "SERVER_PROC" in globals():
    _stop_running(SERVER_PROC)

print("Starting:", " ".join(CMD))
SERVER_PROC = subprocess.Popen(
    CMD,
    cwd=os.environ["WORKDIR"],
    env=ENV,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

def _stream_output(proc):
    for line in proc.stdout:
        print(line, end="")

SERVER_THREAD = threading.Thread(target=_stream_output, args=(SERVER_PROC,), daemon=True)
SERVER_THREAD.start()
print(f"kuso-agent is starting on port {PORT}. 次のセルで UI を開いてください。")


## 6. Colab 上で Web UI を開く

Colab が提供する `serve_kernel_port_as_window` で 8000 番ポートをブラウザに表示します。

In [ ]:
import os

PORT = int(os.environ.get("PORT", "8000"))
try:
    from google.colab import output
    output.serve_kernel_port_as_window(PORT, path="/")
except Exception as err:
    print("Colab 固有の UI 表示に失敗しました。代わりに以下の URL を開いてください:")
    print(f"http://localhost:{PORT}/")
    print(f"詳細: {err}")


## 7. サーバーを停止 (必要に応じて)

作業が終わったらサーバーを停止してリソースを開放します。

In [ ]:
if "SERVER_PROC" in globals() and SERVER_PROC.poll() is None:
    SERVER_PROC.terminate()
    try:
        SERVER_PROC.wait(timeout=10)
    except subprocess.TimeoutExpired:
        SERVER_PROC.kill()
    print("Server stopped.")
else:
    print("Server is not running.")
